In [90]:
import pandas as pd
import geopandas as gpd
import unidecode
import os

path_regioes = "divisoes_regionais.csv" 
path_limites_mun = "Shapefiles_auxiliares\limite_municipal\LimiteMunicipal.shp"
path_PIB = "Processamento_base_PIB_MUN\PIB_processado.csv"
path_ipdm = "Processamento_base_IPDM\ipdm_processada.csv"
path_est_pop = "Processamento_base_EST_POP\EST_POP_processada.csv"
path_regiao_imediata = "Shapefiles_auxiliares\regiao_geografica_imediata\Regiao Geografica Imediata.shp"

### Processamento Individual de cada Base

In [91]:
divisao_regional = pd.read_csv(path_regioes)
divisao_regional = divisao_regional[divisao_regional['Nome_UF'] == 'São Paulo']

divisao_regional.drop(columns=['UF', 'Nome_UF', 'Mesorregião Geográfica',
                      'Nome_Mesorregião', 'Microrregião Geográfica',
                      "Nome_Microrregião", "Município"], inplace=True)

divisao_regional.rename(columns={'Código Município Completo': 'cod_ibge',
                        'Nome_Município': 'Municipio'}, inplace=True)

In [92]:
limites_regionais = gpd.read_file(path_limites_mun)
limites_regionais.drop(columns=['GID_RA','GID_RG','GID_RM', 'GID_RM', 'RM', 'AU', 'GID_AU', 'RA', 'RG'], inplace=True)
limites_regionais.rename(columns={'Cod_ibge': 'cod_ibge',
                                  'Municipios': 'municipio'}, inplace=True)

In [93]:
PIB = pd.read_csv(path_PIB, index_col=0)
# O PIB não possui registros para o ano de 2022
PIB = PIB[PIB['Ano'].isin([2014, 2016, 2018, 2020])]

In [94]:
EST_POP = pd.read_csv(path_est_pop, index_col=0)
EST_POP = EST_POP[EST_POP['Ano'].isin([2014, 2016, 2018, 2020])]
cols = ['razao_sexo', 'id_media', 'dens_demog']
EST_POP[cols] = EST_POP[cols].replace({',': '.'}, regex=True).apply(pd.to_numeric)

In [95]:
ipdm = pd.read_csv(path_ipdm, index_col=0)

### Junção das Bases de acordo com o ano

In [ ]:
def juncao_por_ano(ipdm, limites_regionais, EST_POP, PIB, ano):
    
    caminho_indices_criminais = f"indices_criminais\{ano}\indices_criminais{ano}.csv"
    indices_criminais = pd.read_csv(caminho_indices_criminais)
        
    ipdm_ano = ipdm[ipdm['Ano'] == ano].copy()
    EST_pop_ano = EST_POP[EST_POP['Ano'] == ano].copy()
    PIB_ano = PIB[PIB['Ano'] == ano].copy()
    
    #Padronização dos municipios
    ipdm_ano['Municipio'] = ipdm_ano['Municipio'].apply(lambda x: unidecode.unidecode(x))
    EST_pop_ano['Municipio'] = EST_pop_ano['Municipio'].apply(lambda x: unidecode.unidecode(x))
    PIB_ano['Municipio'] = PIB_ano['Municipio'].apply(lambda x: unidecode.unidecode(x))
    limites_regionais['Municipio'] = limites_regionais['Municipio'].apply(lambda x: unidecode.unidecode(x))
    indices_criminais['Municipio'] = indices_criminais['Municipio'].apply(lambda x: unidecode.unidecode(x))
    
    df_merge1 = pd.merge(ipdm_ano, EST_pop_ano, on=['Ano', 'cod_ibge', 'Municipio'], how='inner')
    df_final = pd.merge(df_merge1, PIB_ano, on=['Ano', 'Municipio'], how='inner')
    df_final = limites_regionais.merge(df_final, on=['cod_ibge', 'Municipio'], how='inner')
    df_final = df_final.merge(indices_criminais, on=['cod_ibge', 'Municipio'], how='inner')
    
    df_final.drop(columns={'Ano_y'}, inplace=True)
    df_final.rename(columns={'Ano_x': 'Ano'}, inplace=True)
    
    return df_final
    
    
df_2014 = juncao_por_ano(ipdm, limites_regionais, EST_POP, PIB, 2014)
df_2016 = juncao_por_ano(ipdm, limites_regionais, EST_POP, PIB, 2016)
df_2018 = juncao_por_ano(ipdm, limites_regionais, EST_POP, PIB, 2018)
df_2020 = juncao_por_ano(ipdm, limites_regionais, EST_POP, PIB, 2020)

In [ ]:
def exportar_df(gdf, ano, caminho_destino='Base_final'):
    diretorio = os.path.join(caminho_destino, str(ano))
    os.makedirs(diretorio, exist_ok=True)
    
    arquivo = os.path.join(diretorio, f"{ano}.shp")
    gdf.to_file(arquivo, driver='ESRI Shapefile')
    print(f"Exportado: {arquivo}")
    
exportar_df(df_2014, 2014)
exportar_df(df_2016, 2016)
exportar_df(df_2018, 2018)
exportar_df(df_2020, 2020)